In [1]:
import pandas as pd
import os
from itertools import permutations

import warnings
warnings.filterwarnings('ignore')


In [2]:
data_dir = 'C:/Users/harry/Documents/data/Project_Node/2021'
report_dir = 'C:/Users/harry/Documents/data/Project_Node/report/'

PeopleCalls = pd.read_csv(os.path.join(data_dir, 'CallParticipants.csv'))
person = pd.read_csv(os.path.join(data_dir, 'PersonHistorical.csv'))
Calls = pd.read_csv(os.path.join(data_dir, 'Calls.csv'))

# Filter Person ID by Internal Netherland Office people 
filtered_person_df = person[(person['Country']=='Netherlands') & (person['WorkerType']=='Employee') & \
                                   (person['Domain']=='CUSHWAKE.COM') & (person['ActiveStatus']=='Yes') & \
                                (person['IsInternal']==True) ]

# Filter Call ID by meeting group size
filtered_calls = Calls[(Calls['TotalParticipants']>2) & ((Calls['TotalParticipants']<6))]


# People call filters by person id and call id
filtered_peoplecalls = PeopleCalls[(PeopleCalls['CallRecordId'].isin(filtered_calls['CallRecordId'])) &
                                   (PeopleCalls['PersonHistoricalId'].isin(filtered_person_df['PersonHistoricalId']))]

# filter People-call by Time Interval 
start_date = pd.to_datetime('2021-12-01')
end_date =  start_date + pd.offsets.MonthBegin(1)
filtered_peoplecalls['LocalStartTime'] = pd.to_datetime(filtered_peoplecalls['LocalStartTime'])
filtered_peoplecalls = filtered_peoplecalls[(filtered_peoplecalls['LocalStartTime'] > start_date) & 
(filtered_peoplecalls['LocalStartTime'] < end_date)]

# filter by duplicated call ids
filtered_peoplecalls = filtered_peoplecalls[filtered_peoplecalls.duplicated(subset='CallRecordId', keep=False)]

In [3]:
filtered_peoplecalls_2021 = filtered_peoplecalls

In [2]:
data_dir = 'C:/Users/harry/Documents/data/Project_Node/2022-08-14-171850/'
report_dir = 'C:/Users/harry/Documents/data/Project_Node/report/'

PeopleCalls = pd.read_csv(os.path.join(data_dir, 'CallParticipants.csv'))
person = pd.read_csv(os.path.join(data_dir, 'PersonHistorical.csv'))
Calls = pd.read_csv(os.path.join(data_dir, 'Calls.csv'))

# Filter Person ID by Internal Netherland Office people 
filtered_person_df = person[(person['Country']=='Netherlands') & (person['WorkerType']=='Employee') & \
                                   (person['Domain']=='CUSHWAKE.COM') & (person['ActiveStatus']=='Yes') & \
                                (person['IsInternal']==True) ]

# Filter Call ID by meeting group size
filtered_calls = Calls[(Calls['TotalParticipants']>2) & ((Calls['TotalParticipants']<6))]


# People call filters by person id and call id
filtered_peoplecalls = PeopleCalls[(PeopleCalls['CallRecordId'].isin(filtered_calls['CallRecordId'])) &
                                   (PeopleCalls['PersonHistoricalId'].isin(filtered_person_df['PersonHistoricalId']))]

# filter People-call by Time Interval 
start_date = pd.to_datetime('2022-02-01')
end_date =  start_date + pd.offsets.MonthBegin(2)
filtered_peoplecalls['LocalStartTime'] = pd.to_datetime(filtered_peoplecalls['LocalStartTime'])
filtered_peoplecalls = filtered_peoplecalls[(filtered_peoplecalls['LocalStartTime'] > start_date) & 
(filtered_peoplecalls['LocalStartTime'] < end_date)]

# filter by duplicated call ids
filtered_peoplecalls = filtered_peoplecalls[filtered_peoplecalls.duplicated(subset='CallRecordId', keep=False)]


In [5]:
filtered_peoplecalls_2022 = filtered_peoplecalls
filtered_peoplecalls = pd.concat([filtered_peoplecalls_2021, filtered_peoplecalls_2022])
filtered_peoplecalls = filtered_peoplecalls[filtered_peoplecalls.duplicated(subset='CallRecordId', keep=False)]


In [3]:
pair_call_df = pd.DataFrame()

for call_id in filtered_peoplecalls['CallRecordId'].unique():
    person_id_list = list(filtered_peoplecalls[filtered_peoplecalls['CallRecordId']==call_id]['PersonHistoricalId'])
    
    temp_df = pd.DataFrame(list(permutations(person_id_list, 2)))
    temp_df.columns = ['PersonId_A', 'PersonId_B']
    temp_df['CallRecordId'] = call_id
    pair_call_df = pd.concat([pair_call_df, temp_df])
    
pair_call_df = pair_call_df.merge(filtered_peoplecalls, on='CallRecordId', how='left')

personid_to_email_dict = filtered_person_df.set_index('PersonHistoricalId')['EmailAddress'].to_dict()
pair_call_df = pair_call_df.replace(personid_to_email_dict)

duration_mins_df = pair_call_df.groupby(by=['PersonId_A', 'PersonId_B']).sum().reset_index().sort_values(by='DurationMinutes', ascending=False)
duration_mins_df.drop(['IsOrganizer'], axis=1, inplace=True)
duration_mins_df.columns = ['EmailAddress1', 'EmailAddress2', 'TotalCallMinutes']

In [4]:
duration_mins_df.to_csv(os.path.join(report_dir, 'Calls_Processed_01022022_31032022.csv'), index=False)

In [5]:
duration_mins_df

,EmailAddress1,EmailAddress2,TotalCallMinutes
3397,F8101C348644FC7A67B97806294C7E720CFC9E605015B7...,1C6FE9A7B6AE883A759F70A626B9F516E4BB48C6AC0A62...,4769.482453
340,1C6FE9A7B6AE883A759F70A626B9F516E4BB48C6AC0A62...,F8101C348644FC7A67B97806294C7E720CFC9E605015B7...,4769.482453
2318,ABF51236B95A952C2876E54557E4753BE827881330AA78...,0F98E80992C5230C56EBEDB2BBC3C921D059206D9E88E8...,4195.265666
206,0F98E80992C5230C56EBEDB2BBC3C921D059206D9E88E8...,ABF51236B95A952C2876E54557E4753BE827881330AA78...,4195.265666
3240,EE6553AAF47DC80910FF3C1578CA7D6B665215A252C89E...,070077EE55B48B57AFAB767AF6CC023576248BCFE9D2D9...,4077.228599
...,...,...,...
2187,A14830DEB743B314488B288513FA288947D69B34F432EC...,C9B9CC53598FD143B9FCA2DD24AD353E3C8F744EB71F0A...,0.139336
1797,872A76F7D43236A4162484F0FD6EB8D870BAD1F9BD443C...,F7B495C1080AEF87DE51598842AA67C2C554CB0040ECA9...,0.134166
3386,F7B495C1080AEF87DE51598842AA67C2C554CB0040ECA9...,872A76F7D43236A4162484F0FD6EB8D870BAD1F9BD443C...,0.134166
2004,93D1C6FD0D72E4F53C2047BC90CF272D539484B60997AE...,EF6B5C134BC0005C124BD37D5322C1737AA29C7DDDCED6...,0.122947
